In [194]:
import pandas as pd
from connection import connect

In [195]:
#connection
SCHEMA='sales'
co_oltp,etl_conn,etl_conn_or=connect(SCHEMA)

['sales_person_quota_history', 'country_region_currency', 'sales_reason', 'credit_card', 'sales_tax_rate', 'currency', 'sales_territory', 'currency_rate', 'sales_territory_history', 'customer', 'shopping_cart_item', 'person_credit_card', 'special_offer', 'special_offer_product', 'sales_order_header_sales_reason', 'sales_order_detail', 'sales_order_header', 'sales_person', 'store']


## Extract

In [196]:
fact_internet_sales=pd.read_sql("""
SELECT  p.product_number, 
sh.order_date, 
sh.due_date, 
sh.ship_date,
sh.territory_id,
so.sales_order_detail_id,
sh.sales_order_number, 
COALESCE(cr.to_currency_code, 'USD') AS to_currency_code,
ROW_NUMBER() OVER (PARTITION BY sh.sales_order_number ORDER BY so.sales_order_detail_id) AS sales_order_line_number,
so.order_qty,
so.unit_price,
(so.unit_price * so.order_qty) AS extended_amount,
so.unit_price_discount,
(so.unit_price * so.unit_price_discount * so.order_qty) AS discount_amount,
p.standard_cost AS product_standard_cost,
(so.order_qty * p.standard_cost) AS total_product_cost,
(so.unit_price *(1-so.unit_price_discount)*so.order_qty) AS sales_amount,
sh.tax_amt,
sh.freight,
so.carrier_tracking_number,
sh.purchase_order_number AS customer_po_number,
sh.order_date,
sh.due_date,
sh.ship_date 



FROM sales.sales_order_header as sh
JOIN sales.sales_order_detail as so
ON sh.sales_order_id=so.sales_order_id
JOIN production.product as p
ON so.product_id=p.product_id
LEFT JOIN sales.currency_rate as cr
ON sh.currency_rate_id=cr.currency_rate_id
WHERE sh.online_order_flag=true
                                
 """,co_oltp)

t_special_offer = pd.read_sql("""
    SELECT p.product_number, sod.special_offer_id
    FROM sales.sales_order_detail AS sod
    JOIN production.product AS p
      ON sod.product_id = p.product_id
    JOIN sales.sales_order_header AS soh
      ON sod.sales_order_id = soh.sales_order_id
    WHERE soh.online_order_flag = true
""", co_oltp)



dim_product=pd.read_sql_table('dim_product',etl_conn)



dim_promotion=pd.read_sql_table('dim_promotion',etl_conn)

dim_sales_territory=pd.read_sql_table('dim_sales_territory',etl_conn)


## Transform

In [197]:
#delete duplicates in t_special_offer and dim_product

dim_product=dim_product.drop_duplicates(subset=['product_alternate_key'])

t_special_offer.drop_duplicates(subset=['product_number'],inplace=True)

In [198]:
# join dim_product

fact_internet_sales=fact_internet_sales.merge(
    dim_product[['product_key','product_alternate_key']],
    left_on='product_number',
    right_on='product_alternate_key',
    how='left'
    
)

fact_internet_sales

,product_number,order_date,due_date,ship_date,territory_id,sales_order_detail_id,sales_order_number,to_currency_code,sales_order_line_number,order_qty,...,sales_amount,tax_amt,freight,carrier_tracking_number,customer_po_number,order_date,due_date,ship_date,product_key,product_alternate_key
0,BK-R93R-62,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,6,353,SO43697,CAD,1,1,...,3578.2700,286.2616,89.4568,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1824,BK-R93R-62
1,BK-M82S-44,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,7,354,SO43698,FRF,1,1,...,3399.9900,271.9992,84.9998,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1860,BK-M82S-44
2,BK-M82S-44,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1,355,SO43699,USD,1,1,...,3399.9900,271.9992,84.9998,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1860,BK-M82S-44
3,BK-R50B-62,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,4,356,SO43700,USD,1,1,...,699.0982,55.9279,17.4775,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1850,BK-R50B-62
4,BK-M82S-44,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,9,357,SO43701,AUD,1,1,...,3399.9900,271.9992,84.9998,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1860,BK-M82S-44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,FE-6654,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,6,121313,SO75122,USD,1,1,...,21.9800,2.4776,0.7743,None,None,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,1999,FE-6654
60394,CA-1098,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,6,121314,SO75122,USD,2,1,...,8.9900,2.4776,0.7743,None,None,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,1737,CA-1098
60395,FE-6654,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,6,121315,SO75123,USD,1,1,...,21.9800,15.1976,4.7493,None,None,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,1999,FE-6654
60396,ST-1401,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,6,121316,SO75123,USD,2,1,...,159.0000,15.1976,4.7493,None,None,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,2000,ST-1401


In [199]:
#join whit dim_product

fact_internet_sales=fact_internet_sales.merge(
    t_special_offer,
    on='product_number',
    how='left'
).drop(['product_number'],axis=1)

fact_internet_sales

,order_date,due_date,ship_date,territory_id,sales_order_detail_id,sales_order_number,to_currency_code,sales_order_line_number,order_qty,unit_price,...,tax_amt,freight,carrier_tracking_number,customer_po_number,order_date,due_date,ship_date,product_key,product_alternate_key,special_offer_id
0,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,6,353,SO43697,CAD,1,1,3578.2700,...,286.2616,89.4568,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1824,BK-R93R-62,1
1,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,7,354,SO43698,FRF,1,1,3399.9900,...,271.9992,84.9998,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1860,BK-M82S-44,1
2,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1,355,SO43699,USD,1,1,3399.9900,...,271.9992,84.9998,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1860,BK-M82S-44,1
3,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,4,356,SO43700,USD,1,1,699.0982,...,55.9279,17.4775,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1850,BK-R50B-62,1
4,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,9,357,SO43701,AUD,1,1,3399.9900,...,271.9992,84.9998,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1860,BK-M82S-44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,6,121313,SO75122,USD,1,1,21.9800,...,2.4776,0.7743,None,None,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,1999,FE-6654,1
60394,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,6,121314,SO75122,USD,2,1,8.9900,...,2.4776,0.7743,None,None,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,1737,CA-1098,1
60395,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,6,121315,SO75123,USD,1,1,21.9800,...,15.1976,4.7493,None,None,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,1999,FE-6654,1
60396,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,6,121316,SO75123,USD,2,1,159.0000,...,15.1976,4.7493,None,None,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,2000,ST-1401,1


In [200]:
#join whit dim_promotion

fact_internet_sales=fact_internet_sales.merge(
    dim_promotion[['promotion_alternate_key','promotion_key']],
    left_on='special_offer_id',
    right_on='promotion_alternate_key',
    how='left'
).drop(['special_offer_id','promotion_alternate_key'],axis=1)

fact_internet_sales



,order_date,due_date,ship_date,territory_id,sales_order_detail_id,sales_order_number,to_currency_code,sales_order_line_number,order_qty,unit_price,...,tax_amt,freight,carrier_tracking_number,customer_po_number,order_date,due_date,ship_date,product_key,product_alternate_key,promotion_key
0,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,6,353,SO43697,CAD,1,1,3578.2700,...,286.2616,89.4568,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1824,BK-R93R-62,1
1,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,7,354,SO43698,FRF,1,1,3399.9900,...,271.9992,84.9998,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1860,BK-M82S-44,1
2,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1,355,SO43699,USD,1,1,3399.9900,...,271.9992,84.9998,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1860,BK-M82S-44,1
3,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,4,356,SO43700,USD,1,1,699.0982,...,55.9279,17.4775,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1850,BK-R50B-62,1
4,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,9,357,SO43701,AUD,1,1,3399.9900,...,271.9992,84.9998,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1860,BK-M82S-44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,6,121313,SO75122,USD,1,1,21.9800,...,2.4776,0.7743,None,None,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,1999,FE-6654,1
60394,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,6,121314,SO75122,USD,2,1,8.9900,...,2.4776,0.7743,None,None,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,1737,CA-1098,1
60395,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,6,121315,SO75123,USD,1,1,21.9800,...,15.1976,4.7493,None,None,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,1999,FE-6654,1
60396,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,6,121316,SO75123,USD,2,1,159.0000,...,15.1976,4.7493,None,None,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,2000,ST-1401,1


In [201]:
# convert dates to integers

fact_internet_sales['order_date']=fact_internet_sales['order_date'].dt.strftime('%Y%m%d').astype(int)
fact_internet_sales['due_date']=fact_internet_sales['due_date'].dt.strftime('%Y%m%d').astype(int)
fact_internet_sales['ship_date']=fact_internet_sales['ship_date'].dt.strftime('%Y%m%d').astype(int)

AttributeError: 'DataFrame' object has no attribute 'dt'

In [ ]:
#rename the columns to propper name in olap database

fact_internet_sales.rename(columns={'order_date':'order_date_key',
                                    'due_date':'due_date_key',
                                    'ship_date':'ship_date_key',    
                                    },inplace=True)

fact_internet_sales

,order_date_key,due_date_key,ship_date_key,territory_id,sales_order_detail_id,sales_order_number,to_currency_code,sales_order_line_number,order_qty,unit_price,...,product_standard_cost,total_product_cost,sales_amount,tax_amt,freight,carrier_tracking_number,customer_po_number,product_key,product_alternate_key,promotion_key
0,20110531,20110612,20110607,6,353,SO43697,CAD,1,1,3578.2700,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,1824,BK-R93R-62,1
1,20110531,20110612,20110607,7,354,SO43698,FRF,1,1,3399.9900,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,1860,BK-M82S-44,1
2,20110531,20110612,20110607,1,355,SO43699,USD,1,1,3399.9900,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,1860,BK-M82S-44,1
3,20110531,20110612,20110607,4,356,SO43700,USD,1,1,699.0982,...,486.7066,486.7066,699.0982,55.9279,17.4775,None,None,1850,BK-R50B-62,1
4,20110531,20110612,20110607,9,357,SO43701,AUD,1,1,3399.9900,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,1860,BK-M82S-44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,20140630,20140712,20140707,6,121313,SO75122,USD,1,1,21.9800,...,8.2205,8.2205,21.9800,2.4776,0.7743,None,None,1999,FE-6654,1
60394,20140630,20140712,20140707,6,121314,SO75122,USD,2,1,8.9900,...,6.9223,6.9223,8.9900,2.4776,0.7743,None,None,1737,CA-1098,1
60395,20140630,20140712,20140707,6,121315,SO75123,USD,1,1,21.9800,...,8.2205,8.2205,21.9800,15.1976,4.7493,None,None,1999,FE-6654,1
60396,20140630,20140712,20140707,6,121316,SO75123,USD,2,1,159.0000,...,59.4660,59.4660,159.0000,15.1976,4.7493,None,None,2000,ST-1401,1


In [ ]:
#join currency

dim_currency=pd.read_sql_table('dim_currency',etl_conn)

fact_internet_sales=fact_internet_sales.merge(
    dim_currency[['currency_key','currency_alternate_key']],
    left_on='to_currency_code',
    right_on='currency_alternate_key',
    how='left'
).drop(['currency_alternate_key','to_currency_code'],axis=1)

fact_internet_sales


,order_date_key,due_date_key,ship_date_key,territory_id,sales_order_detail_id,sales_order_number,sales_order_line_number,order_qty,unit_price,extended_amount,...,total_product_cost,sales_amount,tax_amt,freight,carrier_tracking_number,customer_po_number,product_key,product_alternate_key,promotion_key,currency_key
0,20110531,20110612,20110607,6,353,SO43697,1,1,3578.2700,3578.2700,...,2171.2942,3578.2700,286.2616,89.4568,None,None,1824,BK-R93R-62,1,19
1,20110531,20110612,20110607,7,354,SO43698,1,1,3399.9900,3399.9900,...,1912.1544,3399.9900,271.9992,84.9998,None,None,1860,BK-M82S-44,1,39
2,20110531,20110612,20110607,1,355,SO43699,1,1,3399.9900,3399.9900,...,1912.1544,3399.9900,271.9992,84.9998,None,None,1860,BK-M82S-44,1,100
3,20110531,20110612,20110607,4,356,SO43700,1,1,699.0982,699.0982,...,486.7066,699.0982,55.9279,17.4775,None,None,1850,BK-R50B-62,1,100
4,20110531,20110612,20110607,9,357,SO43701,1,1,3399.9900,3399.9900,...,1912.1544,3399.9900,271.9992,84.9998,None,None,1860,BK-M82S-44,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,20140630,20140712,20140707,6,121313,SO75122,1,1,21.9800,21.9800,...,8.2205,21.9800,2.4776,0.7743,None,None,1999,FE-6654,1,100
60394,20140630,20140712,20140707,6,121314,SO75122,2,1,8.9900,8.9900,...,6.9223,8.9900,2.4776,0.7743,None,None,1737,CA-1098,1,100
60395,20140630,20140712,20140707,6,121315,SO75123,1,1,21.9800,21.9800,...,8.2205,21.9800,15.1976,4.7493,None,None,1999,FE-6654,1,100
60396,20140630,20140712,20140707,6,121316,SO75123,2,1,159.0000,159.0000,...,59.4660,159.0000,15.1976,4.7493,None,None,2000,ST-1401,1,100


In [ ]:
# join sales territoryKey

fact_internet_sales=fact_internet_sales.merge(
    dim_sales_territory[['sales_territory_key','sales_territory_alternate_key']],
    left_on='territory_id',
    right_on='sales_territory_alternate_key',
    how='left'
).drop(['territory_id','sales_territory_alternate_key'],axis=1)

fact_internet_sales





,order_date_key,due_date_key,ship_date_key,sales_order_detail_id,sales_order_number,sales_order_line_number,order_qty,unit_price,extended_amount,unit_price_discount,...,sales_amount,tax_amt,freight,carrier_tracking_number,customer_po_number,product_key,product_alternate_key,promotion_key,currency_key,sales_territory_key
0,20110531,20110612,20110607,353,SO43697,1,1,3578.2700,3578.2700,0.0,...,3578.2700,286.2616,89.4568,None,None,1824,BK-R93R-62,1,19,6
1,20110531,20110612,20110607,354,SO43698,1,1,3399.9900,3399.9900,0.0,...,3399.9900,271.9992,84.9998,None,None,1860,BK-M82S-44,1,39,7
2,20110531,20110612,20110607,355,SO43699,1,1,3399.9900,3399.9900,0.0,...,3399.9900,271.9992,84.9998,None,None,1860,BK-M82S-44,1,100,1
3,20110531,20110612,20110607,356,SO43700,1,1,699.0982,699.0982,0.0,...,699.0982,55.9279,17.4775,None,None,1850,BK-R50B-62,1,100,4
4,20110531,20110612,20110607,357,SO43701,1,1,3399.9900,3399.9900,0.0,...,3399.9900,271.9992,84.9998,None,None,1860,BK-M82S-44,1,6,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,20140630,20140712,20140707,121313,SO75122,1,1,21.9800,21.9800,0.0,...,21.9800,2.4776,0.7743,None,None,1999,FE-6654,1,100,6
60394,20140630,20140712,20140707,121314,SO75122,2,1,8.9900,8.9900,0.0,...,8.9900,2.4776,0.7743,None,None,1737,CA-1098,1,100,6
60395,20140630,20140712,20140707,121315,SO75123,1,1,21.9800,21.9800,0.0,...,21.9800,15.1976,4.7493,None,None,1999,FE-6654,1,100,6
60396,20140630,20140712,20140707,121316,SO75123,2,1,159.0000,159.0000,0.0,...,159.0000,15.1976,4.7493,None,None,2000,ST-1401,1,100,6


In [ ]:
#RevisionNumber

# fact_internet_sales['revision_number']=1



Index(['order_date', 'due_date', 'ship_date', 'territory_id',
       'sales_order_detail_id', 'sales_order_number', 'to_currency_code',
       'sales_order_line_number', 'order_qty', 'unit_price', 'extended_amount',
       'unit_price_discount', 'discount_amount', 'product_standard_cost',
       'total_product_cost', 'sales_amount', 'tax_amt', 'freight',
       'carrier_tracking_number', 'customer_po_number', 'order_date',
       'due_date', 'ship_date', 'product_key', 'product_alternate_key',
       'promotion_key'],
      dtype='object')

In [204]:
#drop unusefull variables

fact_internet_sales.drop(['product_alternate_key'],axis=1,inplace=True)

fact_internet_sales

,order_date,due_date,ship_date,territory_id,sales_order_detail_id,sales_order_number,to_currency_code,sales_order_line_number,order_qty,unit_price,...,sales_amount,tax_amt,freight,carrier_tracking_number,customer_po_number,order_date,due_date,ship_date,product_key,promotion_key
0,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,6,353,SO43697,CAD,1,1,3578.2700,...,3578.2700,286.2616,89.4568,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1824,1
1,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,7,354,SO43698,FRF,1,1,3399.9900,...,3399.9900,271.9992,84.9998,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1860,1
2,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1,355,SO43699,USD,1,1,3399.9900,...,3399.9900,271.9992,84.9998,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1860,1
3,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,4,356,SO43700,USD,1,1,699.0982,...,699.0982,55.9279,17.4775,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1850,1
4,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,9,357,SO43701,AUD,1,1,3399.9900,...,3399.9900,271.9992,84.9998,None,None,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00,1860,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,6,121313,SO75122,USD,1,1,21.9800,...,21.9800,2.4776,0.7743,None,None,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,1999,1
60394,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,6,121314,SO75122,USD,2,1,8.9900,...,8.9900,2.4776,0.7743,None,None,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,1737,1
60395,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,6,121315,SO75123,USD,1,1,21.9800,...,21.9800,15.1976,4.7493,None,None,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,1999,1
60396,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,6,121316,SO75123,USD,2,1,159.0000,...,159.0000,15.1976,4.7493,None,None,2014-06-30 00:00:00+00:00,2014-07-12 00:00:00+00:00,2014-07-07 00:00:00+00:00,2000,1


In [205]:

fact_internet_sales.columns



Index(['order_date', 'due_date', 'ship_date', 'territory_id',
       'sales_order_detail_id', 'sales_order_number', 'to_currency_code',
       'sales_order_line_number', 'order_qty', 'unit_price', 'extended_amount',
       'unit_price_discount', 'discount_amount', 'product_standard_cost',
       'total_product_cost', 'sales_amount', 'tax_amt', 'freight',
       'carrier_tracking_number', 'customer_po_number', 'order_date',
       'due_date', 'ship_date', 'product_key', 'promotion_key'],
      dtype='object')